In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vmlon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vmlon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QFileDialog, QMessageBox, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtGui import QFont, QIcon
from PyQt5.QtCore import QDir, Qt

# Define a global variable to store the CSV file path
csv_file_path = None

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle("CSV File Upload")
        self.setWindowIcon(QIcon('icon.png'))  # Replace 'icon.png' with your icon file
        self.setGeometry(100, 100, 400, 200)
        self.setStyleSheet("""
            QMainWindow {
                background-color: #F0F0F0;
            }
            QPushButton {
                background-color: #4CAF50;
                color: white;
                padding: 10px 20px;
                border-radius: 5px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #45a049;
            }
        """)

        central_widget = QWidget()
        layout = QVBoxLayout()

        open_button = QPushButton("Open CSV")
        open_button.setFont(QFont("Arial", 12))
        open_button.clicked.connect(self.openFile)

        layout.addStretch()
        layout.addWidget(open_button, alignment=Qt.AlignCenter)
        layout.addStretch()

        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

    def openFile(self):
        global csv_file_path
        file_path, _ = QFileDialog.getOpenFileName(self, "Open CSV", QDir.homePath(), "CSV Files (*.csv)")
        if file_path:
            try:
                csv_file_path = file_path  # Store the CSV file path in the global variable
                QMessageBox.information(self, "Success", "CSV file selected successfully.")
                self.close()  # Close the "Open CSV" window

                # You can access the CSV file path using the csv_file_path variable
                # and read the file data using pandas or any other library

            except Exception as e:
                QMessageBox.warning(self, "Error", str(e))

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\vmlon\anaconda3\envs\Project_env\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from flask import Flask ,redirect ,url_for ,request , render_template ,flash

import sqlite3
import requests
import numpy as np
import pandas as pd

# csv_filename = 'Qcsv.csv'
df = pd.read_csv(csv_file_path)
df['QuestionID'] = df.index + 1
print(df)

def GrammerChecker(answer):
    req = requests.get("https://api.textgears.com/check.php?text=" + answer + "&key=JmcxHCCPZ7jfXLF6")
    no_of_errors = len(req.json()['errors'])



    if no_of_errors > 5 :
        g = 0
    else:
        g = 1
    return g


def KeyWordmatching(X, Y_lst):
    result = 0
    X_list = word_tokenize(X)

    sw = stopwords.words('english')
    l1 = []; l2 = []

    X_set = {w for w in X_list if not w in sw}

    for Y in Y_lst:
        Y_list = word_tokenize(Y)
        Y_set = {w for w in Y_list if not w in sw}
        # form a set containing keywords of both strings
        rvector = X_set.union(Y_set)
        if not rvector:
            return 6  # Return worst score if no common keywords
        for w in rvector:
            if w in X_set:
                l1.append(1)  # create a vector
            else:
                l1.append(0)
            if w in Y_set:
                l2.append(1)
            else:
                l2.append(0)
        c = 0

        dot_product = np.dot(l1, l2)
        magnitude_l1 = np.linalg.norm(l1)
        magnitude_l2 = np.linalg.norm(l2)
        cosine = dot_product / (magnitude_l1 * magnitude_l2)

        result += cosine

    cosine = result / 3
    kval = 0
    if cosine > 90:
        kval = 1
    elif cosine > 80:
        kval = 2
    elif cosine > 60:
        kval = 3
    elif cosine > 40:
        kval = 4
    elif cosine > 20:
        kval = 5
    else:
        kval = 6
    return kval



def CheckLenght(client_answer):

    client_ans = len(client_answer.split())

    kval1 = 0
    if client_ans > 50:
        kval1 = 1
    elif client_ans > 40:
        kval1 = 2
    elif client_ans > 30:
        kval1 = 3
    elif client_ans > 20:
        kval1 = 4
    elif client_ans > 10:
        kval1 = 5
    else:
        kval1 = 6
    return kval1


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:

                pass

        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim


    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        result=[]
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            result.append(sim_score)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            results.sort(key=lambda k: k["score"], reverse=True)

        return result



app = Flask(__name__)
email = "null"
name="null"
roll="null"


questions = df.to_dict(orient='records')

@app.route('/')
def Base_qstn_paper_set():
    return render_template('index.html',questions=questions)



@app.route('/foo', methods=['POST', 'GET'])
def foo():
    if request.method == 'POST':
        question_index = int(request.form['question_index'])
        first = request.form['answer{}'.format(question_index)]
        second = request.form['answer{}'.format(question_index + 1)]

        name = request.form['name']
        roll = request.form['roll']
        email = request.form['emailID']
        print(name)
        print(first)

        googlenews_model_path = './GoogleNews-vectors-negative300.bin'
        stopwords_path = "./stopword.txt"


        model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
        with open(stopwords_path, 'r') as fh:
            stopwords = fh.read().split(",")
        ds = DocSim(model,stopwords)
        print("hello")

        def short(source_doc1,target_answer):
            target_docs = [target_answer]
            sim_scores = ds.calculate_similarity(source_doc1, target_docs)
            key_match=KeyWordmatching(source_doc1,target_docs)
            key_Error=GrammerChecker(source_doc1)

            marks1 = max(0, min(50, ((sum(sim_scores) / len(sim_scores)) * 40) + (5/key_match) + (5 * key_Error)))
            return marks1


        def essay(source_doc2,target_answer):
            target_docs = [target_answer]
            sim_scores = ds.calculate_similarity(source_doc2, target_docs)
            key_match=KeyWordmatching(source_doc2,target_docs)
            key_Error=GrammerChecker(source_doc2)
            key_length=CheckLenght(source_doc2)
            marks2 = max(0, min(50, ((sum(sim_scores) / len(sim_scores)) * 30) + (5/key_match) + (5 * key_Error) + (10/key_length)))
            return marks2

        question_row = df.iloc[question_index - 1]
        mark1= short(first,question_row['Answer'])
        mark2= essay(second,question_row['Answer'])
        # mark1=mark1/2
        mark1=round(mark1)
        mark2=round(mark2)
        # mark2=mark2/2
        print(mark1)
        with sqlite3.connect("employee.db") as con:
            cur = con.cursor()
            cur.execute("INSERT into Employees (name, roll, email, marks_short, marks_des ) values (?,?,?,?,?)",(name,roll,email,mark1,mark2))
            con.commit()

            print("After Insert in DB")
            msg = "Employee successfully Added"
    return redirect(url_for('Recorded'))

@app.route('/Recorded')
def Recorded():
    return render_template('base.html')


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 8000, app)

 * Running on http://localhost:8000/ (Press CTRL+C to quit)


                                            Question  \
0                    Define ASPECT OF FUNCTIONALITY.   
1                     Explain in detail INHERITANCE.   
2                        What do you mean by METHOD.   
3  Write a short note on BEHAVIOR FROM ANOTHER CL...   
4           Write a short note on MANAGE COMPLEXITY.   

                                              Answer  QuestionID  
0  Modularity: Modularity is the principle of div...           1  
1  Inheritance also supports the concept of polym...           2  
2  Method overriding allows subclasses to provide...           3  
3  Inheritance: Inheritance is a mechanism that a...           4  
4  By encapsulating related data and behavior wit...           5  
ABC
Modularity: Modularity is the principle of dividing a program into smaller, self-contained modules or classes, each responsible for a specific aspect of functionality
hello
45.0
After Insert in DB


127.0.0.1 - - [23/Apr/2024 22:47:38] "POST /foo HTTP/1.1" 302 -
127.0.0.1 - - [23/Apr/2024 22:47:38] "GET /Recorded HTTP/1.1" 200 -


70
